In [ ]:
options(pkgType = "binary", repos = "https://cran-archive.r-project.org/")

In [ ]:
#install.packages("tidyverse")
#install.packages("rvest")
#install.packages("stringr")
library("tidyverse")
library("rvest")
library("stringr")

In [ ]:
#FUNCIONES
TCD=function(x){
  if(x==1){
    #IMPORTAR EL TIPO DE CAMBIO - si trabajas con el del día
    base_url <- "https://www.sbs.gob.pe/app/pp/sistip_portal/paginas/publicacion/tipocambiopromedio.aspx"
    webpage <- read_html(base_url)
    tipo_de_cambio <- html_nodes(webpage, "td.APLI_fila2:last-child")
    tipo_de_cambio <- as.character(html_text(tipo_de_cambio))
    tipo_de_cambio <- as.numeric(tipo_de_cambio[1])
    tcd=tipo_de_cambio
  }else{
    tcd=as.numeric(readline("Ingrese el tipo de cambio con el que desea trabajar: "))
  }
  return(tcd)
}

percep=function(x){
  if(x==1){
    per=(valor_cif+IGV+IPM)*(3.5/100)
  }else{
    per=0
  }
  return(per)
}

multa=function(x){
  if(x==1){
    print("RESPONER 1 SI LA MONEDA ES EXTRANJERA, 0 DE OTRO MODO")
    moneda_multa=as.numeric(readline("¿En qué moneda está la multa? "))
    if(moneda_multa==1){
      monto_multa=as.numeric(readline("¿Cuánto es el monto de la multa (en dólares)? "))
    }else{
      monto_multa=as.numeric(readline("¿Cuánto es el monto de la multa (en soles)? "))
      monto_multa=monto_multa/tipo_de_cambio_dolares
    }
  }else{
    monto_multa=0
  }
  return(monto_multa)
}


#CÓDIGO
#DATOS DEL PROVEEDOR
proveedor=readline("Ingrese el nombre del proveedor: ")

#CREAR LA TABLA CON LOS VALORES
print("RESPONDER 1 SI ES TIPO DE CAMBIO DEL DÍA, 0 SI ES MANUAL")
tipo_de_cambio_dolares=TCD(as.numeric(readline("¿Desea trabajar con el tipo de cambio del día o ingresarlo manual? ")))
valor_fob=as.numeric(readline("Ingrese el valor FOB (en dólares): "))
flete=as.numeric(readline("Ingrese el flete (en dólares): "))
seguro=as.numeric(readline("Ingrese el valor del seguro (en dólares): "))
valor_cif=valor_fob+flete+seguro
cuadro=data.frame(proveedor, tipo_de_cambio_dolares,valor_fob,flete,seguro,valor_cif)

#cuadro=mutate(cuadro, valor_fob_soles=tipo_de_cambio_dolares*valor_fob)
#cuadro=mutate(cuadro,flete_soles=tipo_de_cambio_dolares*flete)
#cuadro=mutate(cuadro,seguro_soles=tipo_de_cambio_dolares*seguro)
#cuadro=mutate(cuadro,valor_cif_soles=tipo_de_cambio_dolares*valor_cif)

#OTROS COSTOS
#IMPUESTOS
IGV=valor_cif*(16/100)
IPM=valor_cif*(2/100)
print("RESPONDER 1 SI SE PAGA PERCEPCIÓN, 0 DE OTRO MODO")
Percepción=percep(as.numeric(readline("¿Requiere pago de percepción? ")))
total_impuestos=IGV+IPM+Percepción
#ALMACEN
Costo_transporte_almacenje_gastoslocales=as.numeric(readline("Ingrese la suma de gastos locales, almacenaje y transporte (en dólares): "))
#MULTAS - COMISIONES
comisión_interbancaria=as.numeric(readline("Ingresar monto de la comisión interbancaria: "))
print("RESPONDER 1 SI INCURRIÓ EN MULTA, 0 DE OTRO MODO")
monto_multas=multa(as.numeric(readline("¿Incurrió en alguna multa? ")))
#TOTAL OTROS COSTOS
otros_costos=total_impuestos+comisión_interbancaria+monto_multas+Costo_transporte_almacenje_gastoslocales
cuadro=cbind(cuadro,otros_costos)

#COSTO TOTAL
costo_total=round(valor_cif+otros_costos,2)
costo_total_soles=round(costo_total*tipo_de_cambio_dolares,2)
cuadro=cbind(cuadro,costo_total)
cuadro=cbind(cuadro,costo_total_soles)

print(cuadro)
